# Topik Modeling LSA (Latent Semantic Analysis)

Adapun tahapan yang dilakukan untuk melakukan Topik Modeling LSA ada 3 tahapan yakni :
1. Crawling data
    Disini akan dilakuakan crawling data dari website pta ekonomi-manajemen
2. Preprocessing Data
    Data yang telah di crawling akan dilakukan preprocessing guna meningkatkan     nilai dari data
3. LSA (Latent Semantic Analysis)
    Melakukan modeling dengan LSA atau Latent Semantic Analysis

# Crawling Data

Web crawling merupakan salah satu teknik dalam mengumpulkan suatu data yang digunakan untuk mengindekskan suatu informasi yang ada pada halaman menggunakan URL (Uniform Resource Locator) disertai dengan API (Application Programming Interface) dalam melakukan penambangan dataset dengan jumlah yang besar.

Adapun library yang memiliki fungsi untuk membantu proses crawling data yang ada di webstie yakni Scrapy. Berikut ini merupakan cara install library Scrapy. Adapun proses di bawah ini merupakan proses install dari library scrapy.

# Install Library

Library yang digunakan yakni scrapy dan juga nltk.

In [1]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


# Import Library

Setelah berhasil menginstall Scrapy, dapat dilakukan proses crawling data yang ada pada webstie. Pertama dengan mengimportkan library Scrapy yang telah diinstall.

In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess

Setelah mengimportkan beberapa library yang akan digunakan dalam proses crawling. Selanjutnya buat sebuah class **SpiderWeb**, dalam class tersebut berisikan nama dari website yang akan dicrawling disini yakni website "PTA Ekonomi-Manajemen".

In [3]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = []
    def __init__(self):
        url = 'https://pta.trunojoyo.ac.id/welcome/index/'
        for page in range(1,10):
            self.start_urls.append(url+str(page))

    def parse(self, response):
        for detail in response.css('a.gray.button::attr(href)'): 
            yield response.follow(detail.get(), callback = self.parse_detail)

    def parse_detail(self, response):
        for data in response.css('#content_journal > ul > li'):
            yield{
                'Judul': data.css('div:nth-child(2) > a::text').get(),
                'Penulis': data.css('div:nth-child(2) > span::text').get().replace('Penulis : ', ''),
                'Dospem 1': data.css('div:nth-child(3) > span::text').get().replace('Dosen Pembimbing I : ', ''),
                'Dospem 2': data.css('div:nth-child(4) > span::text').get().replace('Dosen Pembimbing II :', ''),
                'Abstraksi': data.css('div:nth-child(2) > p::text').get().replace('\n\n|\n','').replace('ABSTRAK', ''),
                'Abstraction': data.css('div:nth-child(4) > p::text').get().replace('\n\n|\n','').replace('ABSTRACT', ''),
                'Link Download': data.css('div:nth-child(5) > a:nth-child(1)::attr(href)').get().replace('.pdf-0.jpg', '.pdf'),
            }

# Pre-Processing Data

Dalam proses selanjutnya yakni proses Pre-Processing, dalam proses ini memiliki fungsi untuk memastikan kualitas dari data agar data yang digunakan saat analisis data memiliki hasil yang baik dan memiliki data yang clean. Tahapan yang dilakukan untuk Pre-Processing data dilakukan dengan 2 tahap, yakni :
1. Stopword
2. Cleaning Data

# Import Library

Setelah proses install dari 2 library yakni nltk dan scikit-learn,dilakukan proses import library yang digunakan.

In [4]:
import pandas as pd
import numpy as np
import string
import re #regrex libray
import nltk
import swifter
import Sastrawi

from nltk.tokenize import word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

# Load Dataset

Proses berikutnya yakni membuat dan menload dataset.Dalam dataset ini dibuat dengan nama "data_berita" kemudian tambahkan file hasil crawling dengan nama file berikut ini "berita3.csv" 

In [5]:
data_berita = pd.read_csv('crawlUAS.csv')

Lalu tampilkan dataset dengan menggunakan .head() data tertera akan ditampilkan

In [6]:
data_berita.head()

,Judul,Penulis,Dospem 1,Dospem 2,Abstraksi,Abstraction,Link Download
0,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Hendri Wahyudi Prayitno,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Abstrak\r\nPenelitian ini menggunakan metode k...,Abstract\r\nThis research use quantitative met...,https://pta.trunojoyo.ac.id/uploads/journals/0...
1,Pengukuran Website Quality Pada Situs Sistem A...,Muhammad Zakaria Utomo,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Aplikasi nyata pemanfaatan teknologi informasi...,Academic portal system in University of Trunoj...,https://pta.trunojoyo.ac.id/uploads/journals/1...
2,"PENGARUH PERSEPSI MANFAAT, PERSEPSI KEMUDAHAN,...",Ahmad,"Bambang Setiyo Pambudi, S.E., MM.","Fathor AS, S.E., MM.",Tujuan penelitian ini adalah untuk mengidentif...,The purpose of this research was to identify v...,https://pta.trunojoyo.ac.id/uploads/journals/0...
3,PENGARUH PENGEMBANGAN KARIR TERHADAP SEMANGAT ...,Danial Aziz M.YD,"Dr. Hj. Iriani Ismail, Dra.,MM","Drs. Ec. Mudji Kuswinarno, M.Si",PLN merupakan Badan Usaha Milik Negara yang be...,PLN is a state-owned enterprise dealy with the...,https://pta.trunojoyo.ac.id/uploads/journals/0...
4,PENGARUH FAKTOR-FAKTOR STRES KERJA TERHADAP DI...,Saiful Rahman,"Dr. Muhammad Alkirom Wildan,S.E.,M.Si","Helmi Buyung Aulia,S.ST.,S.E.,M.MT",\r\n\r\nPenelitian ini bertujuan untuk mengeta...,\r\n\r\nThis study aimed to determine the effe...,https://pta.trunojoyo.ac.id/uploads/journals/0...


# Tahapan Preprocessing

# 1. Stopword

Stopword removal merupakan tahapan mengambil kata-kata yang penting dari hasil token dengan menggunakan algoritma stoplist (membuang kata yang dinilai kurang penting) atau bisa disebit worldlist (menyimpan kata yang penting). Contoh dari stopword dalam bahasa indonesia yakni sepert "dan","di","dari" dan masih banyak lainnya.

Gunakan library nltk.cotpus dan import stopwords. Seperti dibawah ini

Stopword memiliki additional stopword yang dilist. Jadi ada beberapa kata yang tidak diperlukan dalam data. Kata tersebut dimuat dalam list_stopwords.exted Kemudian dibuat sebuah fungsi dengan nama stopword_removal yang akan membuang kata yang tidak penting di dalam atribut "deskripsi"

In [7]:
def remove_number(text):
    return  re.sub(r"\d+", "", text)

data_berita = data_berita['Abstraksi'].apply(remove_number)
data_berita

0       Abstrak\r\nPenelitian ini menggunakan metode k...
1       Aplikasi nyata pemanfaatan teknologi informasi...
2       Tujuan penelitian ini adalah untuk mengidentif...
3       PLN merupakan Badan Usaha Milik Negara yang be...
4       \r\n\r\nPenelitian ini bertujuan untuk mengeta...
                              ...                        
1014    Umi Kholidah, Pengaruh Arus Kas Operasi, Arus ...
1015    Abstrak\nR.A Fajria Nurul Islami, . Pengaruh S...
1016    \nRiska Utami Ningsih, Pengaruh Kepemilikan in...
1017    Berdasarkan hasil penelitian ini menunjukkan b...
1018    \nLaura Monica, Analisis Pengaruh Likuiditas, ...
Name: Abstraksi, Length: 1019, dtype: object

Langkah sebelum dilakukan stopword diperlukan proses tokenize terlebih dahulu, tokenize adalah sebuah tahapan guna membagi kalimat ke dalam bagian yang telah ditentukan.

In [8]:
word_tokens = data_berita.apply(word_tokenize)
word_tokens

0       [Abstrak, Penelitian, ini, menggunakan, metode...
1       [Aplikasi, nyata, pemanfaatan, teknologi, info...
2       [Tujuan, penelitian, ini, adalah, untuk, mengi...
3       [PLN, merupakan, Badan, Usaha, Milik, Negara, ...
4       [Penelitian, ini, bertujuan, untuk, mengetahui...
                              ...                        
1014    [Umi, Kholidah, ,, Pengaruh, Arus, Kas, Operas...
1015    [Abstrak, R.A, Fajria, Nurul, Islami, ,, ., Pe...
1016    [Riska, Utami, Ningsih, ,, Pengaruh, Kepemilik...
1017    [Berdasarkan, hasil, penelitian, ini, menunjuk...
1018    [Laura, Monica, ,, Analisis, Pengaruh, Likuidi...
Name: Abstraksi, Length: 1019, dtype: object

Langkah selanjutnya yakni melakukan stopword removal dengan menggunakan **stopword.word('indonesia')**

In [9]:
stop_words = stopwords.words('indonesian')
list = ['a','aajaran','aanslag','aatau','ah','abstak','abstrack','abstract','abstrak','z']
stop_words.extend(list)
after = [[w for w in temp if w not in stop_words] for temp in word_tokens]
for i in after:
    print(i)

['Abstrak', 'Penelitian', 'metode', 'kuantitatif', ',', 'menekankan', 'pengujian', 'hipotesis', '.', 'Variabel', 'penelitian', 'kepemimpinan', '(', 'X', ')', ',', 'kompensasi', '(', 'X', ')', '.', 'Penelitian', 'dilaksanakan', 'Kantor', 'Dinas', 'Pendidikan', 'Kabupaten', 'Sampang', 'sampel', 'orang', 'pegawai', 'negeri', 'sipil', 'non', 'PNS', '.', 'Hipotesis', 'variabel', 'bebas', 'bersama–sama', 'hubungan', 'signifikan', 'simultan', 'kepemimpinan', '(', 'X', ')', 'kompensasi', '(', 'X', ')', 'kinerja', 'pegawai', '(', 'Y', ')', ',', 'terbukti', 'nilai', 'Fhitung', 'tingkat', '?', '=', '%', ',', 'nilai', 'R', ',', '.', 'Hipotesis', 'variabel', 'kompensasi', 'variabel', 'pengaruh', 'dominan', ',', 'terbukti', 'nilai', 't', 'hitung', ',', 'pengaruh', 'positif', 'signifikan', 'parsial', 'variabel', 'bebas', '(', 'X', ')', 'variabel', 'terikat', '(', 'Y', ')', '.', 'Kesimpulan', 'penelitian', 'pengujian', 'hipotesis', 'berdasarkan', 'pengujian', 'statistik', 'simultan', ',', 'variabel', 

# Model Menggunakan Topik Modeling LSA

# Install Library

In [10]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

Dalam proses import tersebut ada library yang digunakan dalam visualisasi dan manipulasi data diantaranya yakni :
1. numpy
2. pandas
3. matplotlib
4. seaborn
Serta library yang digunakan dalam pre-processing yakni :
1. nltk

# Load Dataset

Dalam proses modeling juga dilakukan pembuatan dataframe disini data framenya dinamai seperti yang sebelumnya yakni "data_berita". Dan data yang digunakan merupakan data hasil preprocessing dengan nama file "crawlUAS.csv"

In [11]:
data_berita = pd.read_csv('crawlUAS.csv')

Lalu, tampilkan datasetnya dengan menggunakan .head(), dengan hasil seperti berikut ini

In [12]:
data_berita.head()

,Judul,Penulis,Dospem 1,Dospem 2,Abstraksi,Abstraction,Link Download
0,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Hendri Wahyudi Prayitno,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Abstrak\r\nPenelitian ini menggunakan metode k...,Abstract\r\nThis research use quantitative met...,https://pta.trunojoyo.ac.id/uploads/journals/0...
1,Pengukuran Website Quality Pada Situs Sistem A...,Muhammad Zakaria Utomo,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Aplikasi nyata pemanfaatan teknologi informasi...,Academic portal system in University of Trunoj...,https://pta.trunojoyo.ac.id/uploads/journals/1...
2,"PENGARUH PERSEPSI MANFAAT, PERSEPSI KEMUDAHAN,...",Ahmad,"Bambang Setiyo Pambudi, S.E., MM.","Fathor AS, S.E., MM.",Tujuan penelitian ini adalah untuk mengidentif...,The purpose of this research was to identify v...,https://pta.trunojoyo.ac.id/uploads/journals/0...
3,PENGARUH PENGEMBANGAN KARIR TERHADAP SEMANGAT ...,Danial Aziz M.YD,"Dr. Hj. Iriani Ismail, Dra.,MM","Drs. Ec. Mudji Kuswinarno, M.Si",PLN merupakan Badan Usaha Milik Negara yang be...,PLN is a state-owned enterprise dealy with the...,https://pta.trunojoyo.ac.id/uploads/journals/0...
4,PENGARUH FAKTOR-FAKTOR STRES KERJA TERHADAP DI...,Saiful Rahman,"Dr. Muhammad Alkirom Wildan,S.E.,M.Si","Helmi Buyung Aulia,S.ST.,S.E.,M.MT",\r\n\r\nPenelitian ini bertujuan untuk mengeta...,\r\n\r\nThis study aimed to determine the effe...,https://pta.trunojoyo.ac.id/uploads/journals/0...


Sebelum beralih pada tahap selanjutnya, ada beberapa atribut yang perlu dihapus yakni judul, tanggal, dan kategori. Dikarenakan atribut yang digunakan yakni atribut deskripsi.

In [13]:
# drop judul,penulis, dospem 1 & 2
data_berita.drop(['Judul', 'Penulis', 'Dospem 1', 'Dospem 2','Abstraction', 'Link Download'],axis=1,inplace=True)

Tampilkan data yang telah di hapus atributnya dan menyisakan deskrips

In [14]:
data_berita.head()

,Abstraksi
0,Abstrak\r\nPenelitian ini menggunakan metode k...
1,Aplikasi nyata pemanfaatan teknologi informasi...
2,Tujuan penelitian ini adalah untuk mengidentif...
3,PLN merupakan Badan Usaha Milik Negara yang be...
4,\r\n\r\nPenelitian ini bertujuan untuk mengeta...


# Data Cleaning

Pada tahap selnajutnya yakni melakukan cleaning dan juga tahap pre-processing kembali. Menggunakan lemmatizer danjuga stemmer. Terdapat juga kata-kata yang tidak digubakan bersama dan juga kata-kata yang panjanya kurang dari 3 karakter yang berguna untuk mengurangi beberapa kata yang tidak cocok.

In [15]:
def clean_text(headline):
  le=WordNetLemmatizer()
  word_tokens=word_tokenize(headline)
  tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
  cleaned_text=" ".join(tokens)
  return cleaned_text

In [17]:
# time taking
data_berita['Abstraksi_cleaned_text']=data_berita['Abstraksi'].apply(clean_text)

In [18]:
data_berita.head()

,Abstraksi,Abstraksi_cleaned_text
0,Abstrak\r\nPenelitian ini menggunakan metode k...,Abstrak Penelitian menggunakan metode kuantita...
1,Aplikasi nyata pemanfaatan teknologi informasi...,Aplikasi nyata pemanfaatan teknologi informasi...
2,Tujuan penelitian ini adalah untuk mengidentif...,Tujuan penelitian adalah untuk mengidentifikas...
3,PLN merupakan Badan Usaha Milik Negara yang be...,merupakan Badan Usaha Milik Negara yang berger...
4,\r\n\r\nPenelitian ini bertujuan untuk mengeta...,Penelitian bertujuan untuk mengetahui pengaruh...


Dapat dilihat perbedaan setelah dilakukan stopwords dan juga kata yang lebih pendek. Juga ada beberapa kata yang telah diestimasi seperti "calls"-->"call"

In [20]:
data_berita.drop(['Abstraksi'],axis=1,inplace=True)

In [21]:
data_berita.head()

,Abstraksi_cleaned_text
0,Abstrak Penelitian menggunakan metode kuantita...
1,Aplikasi nyata pemanfaatan teknologi informasi...
2,Tujuan penelitian adalah untuk mengidentifikas...
3,merupakan Badan Usaha Milik Negara yang berger...
4,Penelitian bertujuan untuk mengetahui pengaruh...


In [22]:
data_berita['Abstraksi_cleaned_text'][0]

'Abstrak Penelitian menggunakan metode kuantitatif yang menekankan pada pengujian hipotesis Variabel yang digunakan pada penelitian adalah kepemimpinan kompensasi Penelitian dilaksanakan Kantor Dinas Pendidikan Kabupaten Sampang sampel yang digunakan adalah sebanyak orang yang terdiri dari pegawai negeri sipil Hipotesis pertama menjelaskan bahwa variabel bebas secara bersama–sama mempunyai hubungan yang signifikan secara simultan antara kepemimpinan kompensasi terhadap kinerja pegawai terbukti dengan nilai Fhitung pada tingkat sebesar 11,645 dengan nilai sebesar 0,311 Hipotesis kedua menjelaskan bahwa variabel kompensasi sebagai variabel yang mempunyai pengaruh dominan terbukti dengan nilai hitung 3,072 berarti bahwa terdapat pengaruh positif signifikan secara parsial antara variabel bebas terhadap variabel terikat Kesimpulan penelitian adalah pengujian hipotesis pertama menunjukkan bahwa berdasarkan pengujian statistik secara simultan variabel bebas memiliki pengaruh yang signifikan t

Dapat dilihat bahwa ada beberapa judul berita tertentu

# Extracting Feature dan Membuat Dokumen Term-Matrik (DTM)

DTM memiliki nilai TF-Idf Tentukan pula parameter dari vectore TF-Idf

Adapun beberapa poin penting yang perlu diperhatoakn :

LSA pada umumnua diimplementasikan dengan menggunakan nilai TF-Idff dan tidak dengan Count Vectorizer
max_feature bergantubg pada daya komputasi dan juga pada eval. Metrik merupakan skor yang koheren untuk menentukan model
Nilai default untuk min_df dan max_df dapat bekerja dengan baik
Dapat mencoba nilai yang berbeda-beda dalam ngram_range

In [23]:
vect =TfidfVectorizer(stop_words=stop_words, max_features=1000) # to play with. min_df,max_df,max_features etc...

In [25]:
vect_text=vect.fit_transform(data_berita['Abstraksi_cleaned_text'])

Dalam hasilnya, dapat dilihat kata yang sering muncul dan kata yang jarang muncul dalam berita yang ada dalam idf. Apabila memiliki nilai yang kecil maka katanya lebih umum digunakan dalam berita utama

In [26]:
print(vect_text.shape)
print(vect_text)

(1019, 1000)
  (0, 467)	0.022635638327625874
  (0, 391)	0.02250178030957979
  (0, 122)	0.024709956136154426
  (0, 573)	0.047881691746207496
  (0, 536)	0.0815690546806959
  (0, 539)	0.049454663453802124
  (0, 889)	0.06982400165883748
  (0, 114)	0.033929390887651095
  (0, 566)	0.030163446648598238
  (0, 432)	0.0700652185554749
  (0, 935)	0.12399280391183405
  (0, 629)	0.03208302044532638
  (0, 732)	0.035790573500601434
  (0, 932)	0.05688763841434733
  (0, 112)	0.06074243442421942
  (0, 320)	0.07379075638406334
  (0, 243)	0.08694593454547808
  (0, 668)	0.07420444737000494
  (0, 821)	0.05079074567243407
  (0, 400)	0.14700047616227604
  (0, 7)	0.09074990080207347
  (0, 825)	0.06661492114827604
  (0, 950)	0.045912771911561526
  (0, 289)	0.06265431808451072
  (0, 597)	0.10445273238487683
  :	:
  (1018, 189)	0.031199481834644865
  (1018, 56)	0.041946309736188445
  (1018, 972)	0.012027939310659598
  (1018, 171)	0.029026739528994892
  (1018, 467)	0.012924654007991324
  (1018, 391)	0.012848222827

In [27]:
idf=vect.idf_

In [29]:
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])
print(dd['penelitian'])
print(dd['need'])  # police is most common and forecast is least common among the news headlines.

penelitian need
1.0068864020296335
7.234410725718371


Oleh karena itu, berdasarkan pada nilai idf yang ada 'penelitian' adalah kata yang paling sering muncul sedangkan 'need' paling jarang muncul dalam berita

# Topik Modeling LSA

Pada pendekatan pertama digunakan LSA. LSA pada dasarnya adalah dekomposisi dari nilai tunggal.

SVD akan menguraikan DTM menajdi tiga matriks = S=U.(sigma).(V.T). Nilai matrik U menunjukkan matriks dari dokumen topik sementara (V) adalah matriks dari term.

Pada setiap baris dari matriks U (matriks istilah dari dokumen) merupakan representasi vektor yang ada dalam dokumen yang sesuai. Panjang vektor ini ialah jumlah topik yang diinginkan. Representasi dari vektor untuk suku yang ada dalam data dapat ditemui dalam matriks V.

Jadi, SVD memberikan nilai vektor pada setiap dokumen dan juga istilah dalam data. Panjang dari setiap vektor adalah k. Vektor ini digunakan untuk menentukan kata dan dokumen serupa dalam metode kesamaan kosinus.

Dapat digunakan fungsi truncastedSVD untuk mengimplementasikan LSA. Parameter n_components merupakan jumlah topik yang akan diekstrak. Model tersebut nantinya akan di fit dan ditransformasikan pada hasil yang diberikan oleh vectorizer.

Tahap terakhir yakni LSA dan LSI (I digunakan untuk mengindekskan) ialah sama dan yang terakhir digunakan dalam konteks pencarian sebuah informasi

In [30]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)

In [33]:
print(lsa_top)
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

[[ 0.40229073  0.05549972  0.03121153 ... -0.13722768 -0.01490652
  -0.13806903]
 [ 0.16816737 -0.00672889  0.09201281 ... -0.02885376  0.09588651
   0.08748818]
 [ 0.18972676 -0.0473179   0.0910802  ... -0.00640613  0.02285941
  -0.00725145]
 ...
 [ 0.26804609 -0.197341   -0.09325366 ... -0.10329699  0.04125433
   0.0049161 ]
 [ 0.32953475 -0.27080198 -0.14398105 ... -0.13609479  0.09508535
  -0.05079068]
 [ 0.22953822 -0.17968864 -0.0895312  ...  0.26774633 -0.12195237
   0.04602788]]
(1019, 10)


In [34]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  40.22907313025696
Topic  1  :  5.549972023821379
Topic  2  :  3.1211532563943503
Topic  3  :  -2.370151091503758
Topic  4  :  -18.641942375784335
Topic  5  :  -6.6486140510212
Topic  6  :  12.639603165005436
Topic  7  :  -13.722768201765465
Topic  8  :  -1.4906521496100782
Topic  9  :  -13.806902960178014


Hampir sama dengan dokumen lainnya, dapat dilakukan proses seperti di bawah ini. Akan tetapi perlu diperhatikan bahwa dalam setiap nilai tidak menambah 1 itu bukan sebuah kemungkinan topik yang ada dalam dokumen.

In [35]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 1000)
[[ 0.05154667  0.00835509  0.00626683 ...  0.00563758  0.00432133
   0.00748422]
 [ 0.04152337  0.00365113  0.00222593 ... -0.00605957 -0.00827659
  -0.0153578 ]
 [-0.01940661 -0.0040024  -0.00168403 ...  0.01488802 -0.0055252
  -0.01176884]
 ...
 [-0.09926132 -0.01032602 -0.00870884 ...  0.00188085  0.01759257
   0.00209193]
 [-0.10817699 -0.01190601 -0.01819463 ... -0.0017663  -0.01196481
  -0.004904  ]
 [ 0.08643873  0.01072942  0.01006749 ... -0.00587757  0.00704055
   0.00600814]]


Sehingga, didapatkan sebuah list dari kata-kata yang penting dan memiliki makna dari setiap 10 topic yang ditampilkan. Sederhananya dibawah ini ditampilkan 10 kata dalam setiap topic.

In [36]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
kerja karyawan terhadap yang kinerja variabel perusahaan penelitian berpengaruh sebesar 

Topic 1: 
kerja karyawan kinerja motivasi kepuasan disiplin lingkungan sebesar kompensasi organisasi 

Topic 2: 
keputusan pembelian produk merek konsumen kualitas promosi citra variabel mahasiswa 

Topic 3: 
ratio saham return equity harga debt earning share current kerja 

Topic 4: 
kerja kepuasan prestasi stres kualitas struktur produktivitas pelayanan perusahaan modal 

Topic 5: 
saham stock split harga volume sesudah sebelum perdagangan perbedaan yang 

Topic 6: 
kepuasan bank pelayanan pegawai pelanggan kualitas yang loan rasio empati 

Topic 7: 
bank rasio keuangan merek kerja kinerja karyawan ratio citra loan 

Topic 8: 
karyawan kepuasan kinerja pelayanan komitmen kualitas kompensasi pelanggan harga variabel 

Topic 9: 
merek kepuasan citra karyawan kualitas nilai sebesar 05 brand pelayanan 



# Pre-Processing

# Load Dataset

In [42]:
data_berita = pd.read_csv('crawlUAS.csv')

# Case Folding

Case folding adalag proses pre-processing yang memiliki fungsi untuk menyeragamkan karakter pada data yang ada. Proses Case Folding merupakan proses dalam mengubah huruf kecil. Pada karakter "A-Z" menjadi "a-z"

In [45]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data_berita['Abstraksi'] = data_berita['Abstraksi'].str.lower()


print('Case Folding Result : \n')
print(data_berita['Abstraksi'].head(20))
print('\n\n\n')

Case Folding Result : 

0     abstrak\r\npenelitian ini menggunakan metode k...
1     aplikasi nyata pemanfaatan teknologi informasi...
2     tujuan penelitian ini adalah untuk mengidentif...
3     pln merupakan badan usaha milik negara yang be...
4     \r\n\r\npenelitian ini bertujuan untuk mengeta...
5     munculnya asumsi bahwa lahirnya budaya mengemi...
6     muhammad sholeh, pengaruh iklan di televisi te...
7     atribut produk memberikan pandangan atau gamba...
8     ulfiyatun mutohharoh, “analisis faktor-faktor ...
9     \r\ntujuan penelitian ini adalah untuk mengana...
10    anissa novianti. 090211100010. pengaruh atribu...
11    devi kurniawati, pengaruh earning per share, n...
12    \r\ntujuan penelitian ini adalah (1) untuk men...
13    tujuan penelitian ini adalah untuk mengetahui ...
14    dwi suhardianto (2013), “pengaruh karakteristi...
15    farid,  pengaruh  periklanan di media televisi...
16    abstrak\r\n\r\npengaruh kecerdasan emosional t...
17    \r\n\r\npenelitian

Digunakan fungsi series.str.lower() yang terdapat pandas. Atribut yang dilakukan proses case folding yakni "Abstraksi"

# Tokenizing

Tokenizing merupakan tahap pemotongan string yang berdasarkan kata-kata yang menyusunkan atau memcahkan kalimat menjadi sebuah kata.Dalam tahap tokenizing yang pertama import string dan juga mengimport library re. Dan juga diperlukan library yang digunakan untuk memproses tokenize yakni nltk.tokenize dan nltk.probability

In [46]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [48]:
def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return re.sub(r"\d+", "", text)

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data_berita['Abstraksi_tokens'] = data_berita['Abstraksi'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data_berita['Abstraksi_tokens'].head(20))
print('\n\n\n')

Tokenizing Result : 

0     [abstrak, penelitian, ini, menggunakan, metode...
1     [aplikasi, nyata, pemanfaatan, teknologi, info...
2     [tujuan, penelitian, ini, adalah, untuk, mengi...
3     [pln, merupakan, badan, usaha, milik, negara, ...
4     [penelitian, ini, bertujuan, untuk, mengetahui...
5     [munculnya, asumsi, bahwa, lahirnya, budaya, m...
6     [muhammad, sholeh, pengaruh, iklan, di, televi...
7     [atribut, produk, memberikan, pandangan, atau,...
8     [ulfiyatun, mutohharoh, analisis, faktorfaktor...
9     [tujuan, penelitian, ini, adalah, untuk, menga...
10    [anissa, novianti, pengaruh, atribut, produk, ...
11    [devi, kurniawati, pengaruh, earning, per, sha...
12    [tujuan, penelitian, ini, adalah, untuk, menge...
13    [tujuan, penelitian, ini, adalah, untuk, menge...
14    [dwi, suhardianto, pengaruh, karakteristik, in...
15    [farid, pengaruh, periklanan, di, media, telev...
16    [abstrak, pengaruh, kecerdasan, emosional, ter...
17    [penelitian, ini, be

# Menghitung Frekuensi Distribusi Token

In [50]:
# NLTK calc frequency distribution
def freqDist_wrapper(text):
    return FreqDist(text)

data_berita['Abstraksi_tokens_fdist'] = data_berita['Abstraksi_tokens'].apply(freqDist_wrapper)

print('Frequency Tokens : \n') 
print(data_berita['Abstraksi_tokens_fdist'].head(20).apply(lambda x : x.most_common()))

Frequency Tokens : 

0     [(variabel, 10), (yang, 8), (ini, 7), (hipotes...
1     [(dan, 8), (akademik, 8), (portal, 5), (kepuas...
2     [(menggunakan, 6), (persepsi, 6), (internet, 5...
3     [(yang, 7), (pln, 5), (penelitian, 5), (kerja,...
4     [(stressor, 9), (kerja, 6), (disiplin, 4), (ya...
5     [(yang, 13), (budaya, 9), (pengemis, 6), (yait...
6     [(pesan, 9), (terhadap, 6), (pengaruh, 5), (un...
7     [(produk, 20), (motor, 12), (yang, 8), (honda,...
8     [(yang, 16), (produk, 10), (online, 9), (fakto...
9     [(jembatan, 5), (suramadu, 5), (dan, 4), (peng...
10    [(kamal, 14), (dan, 10), (atribut, 8), (produk...
11    [(dan, 10), (yang, 7), (terhadap, 6), (harga, ...
12    [(pesan, 18), (dan, 6), (minat, 6), (beli, 6),...
13    [(yang, 14), (citra, 10), (image, 8), (penelit...
14    [(dan, 15), (yang, 13), (prestasi, 7), (kerja,...
15    [(pesan, 9), (pengaruh, 5), (yang, 5), (terhad...
16    [(kecerdasan, 8), (kerja, 7), (dan, 7), (prest...
17    [(merek, 9), (dan, 9)

# Filtering (Stopword Removal)

Stopword removal merupakan tahapan mengambil kata-kata yang penting dari hasil token dengan menggunakan algoritma stoplist (membuang kata yang dinilai kurang penting) atau bisa disebit worldlist (menyimpan kata yang penting). Contoh dari stopword dalam bahasa indonesia yakni sepert "dan","di","dari" dan masih banyak lainnya.

Gunakan library nltk.cotpus dan import stopwords. Seperti dibawah ini

Stopword memiliki additional stopword yang dilist. Jadi ada beberapa kata yang tidak diperlukan dalam data. Kata tersebut dimuat dalam list_stopwords.exted Kemudian dibuat sebuah fungsi dengan nama stopword_removal yang akan membuang kata yang tidak penting di dalam atribut "deskripsi"

In [51]:
from nltk.corpus import stopwords
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("crawlUAS.csv", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data_berita['Abstraksi_tokens_WSW'] = data_berita['Abstraksi_tokens'].apply(stopwords_removal) 


print(data_berita['Abstraksi_tokens_WSW'].head(20))

0     [abstrak, penelitian, metode, kuantitatif, men...
1     [aplikasi, nyata, pemanfaatan, teknologi, info...
2     [tujuan, penelitian, mengidentifikasi, variabe...
3     [pln, badan, usaha, milik, negara, bergerak, b...
4     [penelitian, bertujuan, pengaruh, faktorfaktor...
5     [munculnya, asumsi, lahirnya, budaya, mengemis...
6     [muhammad, sholeh, pengaruh, iklan, televisi, ...
7     [atribut, produk, pandangan, gambaran, produk,...
8     [ulfiyatun, mutohharoh, analisis, faktorfaktor...
9     [tujuan, penelitian, menganalisis, atribut, ku...
10    [anissa, novianti, pengaruh, atribut, produk, ...
11    [devi, kurniawati, pengaruh, earning, share, n...
12    [tujuan, penelitian, pengaruh, pesan, iklan, t...
13    [tujuan, penelitian, pengaruh, variabel, brand...
14    [dwi, suhardianto, pengaruh, karakteristik, in...
15    [farid, pengaruh, periklanan, media, televisi,...
16    [abstrak, pengaruh, kecerdasan, emosional, pre...
17    [penelitian, bertujuan, pengaruh, citra, m

# Normalization

Normalization merupakan salah satu tahapan yang ada dalam pre-processing. Normalization memiliki fungsi untuk menyamakan nilai yang ada dalam data dalam rentan tertentu.

In [52]:
normalizad_word = pd.read_csv("crawlUAS.csv")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

data_berita['Abstraksi_normalized'] = data_berita['Abstraksi_tokens_WSW'].apply(normalized_term)

data_berita['Abstraksi_normalized'].head(20)

0     [abstrak, penelitian, metode, kuantitatif, men...
1     [aplikasi, nyata, pemanfaatan, teknologi, info...
2     [tujuan, penelitian, mengidentifikasi, variabe...
3     [pln, badan, usaha, milik, negara, bergerak, b...
4     [penelitian, bertujuan, pengaruh, faktorfaktor...
5     [munculnya, asumsi, lahirnya, budaya, mengemis...
6     [muhammad, sholeh, pengaruh, iklan, televisi, ...
7     [atribut, produk, pandangan, gambaran, produk,...
8     [ulfiyatun, mutohharoh, analisis, faktorfaktor...
9     [tujuan, penelitian, menganalisis, atribut, ku...
10    [anissa, novianti, pengaruh, atribut, produk, ...
11    [devi, kurniawati, pengaruh, earning, share, n...
12    [tujuan, penelitian, pengaruh, pesan, iklan, t...
13    [tujuan, penelitian, pengaruh, variabel, brand...
14    [dwi, suhardianto, pengaruh, karakteristik, in...
15    [farid, pengaruh, periklanan, media, televisi,...
16    [abstrak, pengaruh, kecerdasan, emosional, pre...
17    [penelitian, bertujuan, pengaruh, citra, m

In [53]:

# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data_berita['Abstraksi_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data_berita['Abstraksi_tokens_stemmed'] = data_berita['Abstraksi_normalized'].swifter.apply(get_stemmed_term)
print(data_berita['Abstraksi_tokens_stemmed'])

7548
------------------------
abstrak : abstrak
penelitian : teliti
metode : metode
kuantitatif : kuantitatif
menekankan : tekan
pengujian : uji
hipotesis : hipotesis
variabel : variabel
kepemimpinanx : kepemimpinanx
kompensasi : kompensasi
dilaksanakan : laksana
kantor : kantor
dinas : dinas
pendidikan : didik
kabupaten : kabupaten
sampang : sampang
sampel : sampel
orang : orang
pegawai : pegawai
negeri : negeri
sipil : sipil
non : non
pns : pns
bebas : bebas
bersamasama : bersamasama
hubungan : hubung
signifikan : signifikan
simultan : simultan
kepemimpinan : pimpin
kinerja : kerja
terbukti : bukti
nilai : nilai
fhitung : fhitung
tingkat : tingkat
pengaruh : pengaruh
dominan : dominan
hitung : hitung
positif : positif
parsial : parsial
terikat : ikat
kesimpulan : simpul
berdasarkan : dasar
statistik : statistik
memiliki : milik
membuktikan : bukti
berpengaruh : pengaruh
kunci : kunci
aplikasi : aplikasi
nyata : nyata
pemanfaatan : manfaat
teknologi : teknologi
informasi : informasi
k

galis : galis
blega : blega
memilki : memilki
memahami : paham
hati : hati
menciptakan : cipta
suasana : suasana
nyaman : nyaman
meningkatkan : tingkat
peningkatan : tingkat
kerjanya : kerja
meneliti : teliti
intrapersonal : intrapersonal
interpersonal : interpersonal
linear : linear
bantuan : bantu
kuncikecerdasan : kuncikecerdasan
vario : vario
keunggulan : unggul
asosiasi : asosiasi
kekuatan : kuat
keunikan : uni
produktifitas : produktifitas
dibutuhkan : butuh
mewujudkan : wujud
visi : visi
misi : misi
produktivitas : produktivitas
ditingkatkan : tingkat
anggota : anggota
terlaksananya : laksana
optimal : optimal
terkait : kait
kefektifan : kefektifan
berkomunikasi : komunikasi
saluran : salur
kebawah : bawah
keatas : atas
horizontal : horizontal
meningkatnya : tingkat
kali : kali
primer : primer
penyebaran : sebar
kuisoner : kuisoner
respondennya : responden
starfood : starfood
international : international
paciran : paciran
lamongan : lamongan
clustered : clustered
analisi : anal

syahkesimpulan : syahkesimpulan
tipe : tipe
autoritarian : autoritarian
laissezfaire : laissezfaire
demokratis : demokratis
pangarengan : pangarengan
anova : anova
kontribusi : kontribusi
autoritariansecara : autoritariansecara
pegawaidikantor : pegawaidikantor
laissezfairesecara : laissezfairesecara
lainya : lai
dikantor : kantor
haryono : haryono
arifin : arifin
dealler : dealler
dianalisa : dianalisa
aththaariq : aththaariq
kompetensi : kompetensi
drrm : drrm
moch : moch
wispandonosems : wispandonosems
wildansemsi : wildansemsi
pedagogik : pedagogik
profesional : profesional
kepribadian : pribadi
tersertifikasi : sertifikasi
judgmental : judgmental
koefisiensi : koefisiensi
kepribadiandan : kepribadiandan
retailing : retailing
merchandise : merchandise
atmosfer : atmosfer
gerai : gerai
retailimg : retailimg
idola : idola
mart : mart
kwanyar : kwanyar
rumusan : rumus
pelaporan : lapor
meminimalkan : minimal
cacat : cacat
cv : cv
kapuas : kapuas
inti : inti
sarana : sarana
berasal : a

terstruktur : struktur
samsuki : samsuki
iklim : iklim
bumn : bumn
calon : calon
lingkungannya : lingkung
mendorong : dorong
pelayan : layan
nasabahnya : nasabah
dilanjutkan : lanjut
imbalan : imbal
kehangatan : hangat
simutan : simutan
reny : reny
nurhalimah : nurhalimah
added : added
sektor : sektor
farmasi : farmasi
echsan : echsan
gani : gani
berinvestasi : investasi
mva : mva
kekayaan : kaya
dihasilkan : hasil
penghasilan : hasil
pemegang : pegang
pemilik : milik
prospek : prospek
tingginya : tinggi
sensitivitas : sensitivitas
mita : mita
anggun : anggun
pratiwi : pratiwi
kosmetik : kosmetik
wardah : wardah
muhamad : muhamad
penelitin : penelitin
ditimbulkan : timbul
sosail : sosail
mahasiswi : mahasiswi
makan : makan
lesehan : leseh
berkah : berkah
ilaahi : ilaahi
menganti : anti
faiqotul : faiqotul
muna : muna
responsibility : responsibility
semen : semen
sudarsono : sudarsono
level : level
significance : significance
linieritas : linieritas
csr : csr
milyar : milyar
sumirah : s

hadiatullah : hadiatullah
perusahaanperusahaan : perusahaanperusahaan
diinterpretasikan : interpretasi
hanjaya : hanjaya
internasional : internasional
lancar : lancar
earned : earned
fixed : fixed
lukman : lukman
waqi : waqi
arianto : arianto
publik : publik
mahkmud : mahkmud
mengandung : kandung
pastian : pasti
dikehendaki : hendak
mengkombinasikan : kombinasi
sekuritas : sekuritas
tercatat : catat
standart : standart
deviasi : deviasi
kombinasi : kombinasi
trada : trada
maritime : maritime
ambil : ambil
potofolio : potofolio
pleh : pleh
diversifikasi : diversifikasi
refti : refti
selvi : selvi
wahyuni : wahyuni
pajak : pajak
pratama : pratama
rm : rm
mochammad : mochammad
wispandono : wispandono
sems : sems
kpp : kpp
institusi : institusi
penggerak : gerak
penentu : tentu
tercapainya : capa
inovatif : inovatif
berpengruh : ruh
muazzah : muazzah
judul : judul
overreaction : overreaction
dosem : dosem
decade : decade
perdagangkan : dagang
penyimpangan : simpang
januari : januari
effect

...

pmii : pmii
arno : arno
mengekplorasi : mengekplorasi
rayon : rayon
asshiddiq : asshiddiq
ikamapaba : ikamapaba
istrumen : istrumen
backup : backup
ditarik : tarik
mempertahan : tahan
anny : anny
pangistu : pangistu
ning : ning
pensiun : pensiun
ptpelindo : ptpelindo
karyawanteknik : karyawanteknik
pension : pension
mental : mental
spritual : spritual
tersebutoleh : tersebutoleh
mencapat : mencapat
indifidu : indifidu
organisasikemitraan : organisasikemitraan
kemitraan : mitra
oisau : oisau
deskriptifwawancara : deskriptifwawancara
tahapan : tahap
analisisyaitu : analisisyaitu
kategorisasi : kategorisasi
sitesisasi : sitesisasi
poin : poin
forma : forma
pik : pik
galaxi : galaxi
lansia : lansia
klinik : klinik
nu : nu
kb : kb
formalnya : formal
paming : paming
mamfaat : mamfaat
jangkauan : jangkau
ranah : ranah
remaja : remaja
mudahterlebih : mudahterlebih
kimitraan : kimitraan
penyuluhan : suluh
pendampingan : damping
penanggulangan : tanggulang
kemitraanmanajemen : kemitraanmanajemen

skillberpengaruhpositifdansignifikanterhadapproduktivitaskaryawan : skillberpengaruhpositifdansignifikanterhadapproduktivitaskaryawan
sedangkanuntukvariabelsoft : sedangkanuntukvariabelsoft
didapatkanhasilkoefisienregresi : didapatkanhasilkoefisienregresi
dengannilaisignifikansi : dengannilaisignifikansi
lebihkecildaritingkatsignifikan : lebihkecildaritingkatsignifikan
untuknilaithitungttabel : untuknilaithitungttabel
inijugamenunjukkanbahwavariabel : inijugamenunjukkanbahwavariabel
soft : soft
berpengaruhpositifdansignifikanterhadapproduktifitaskaryawan : berpengaruhpositifdansignifikanterhadapproduktifitaskaryawan
secarasimultanvariabelhard : secarasimultanvariabelhard
skilljugaberpengaruhpositifterhadapproduktivitaskaryawandengannilaifhitungf : skilljugaberpengaruhpositifterhadapproduktivitaskaryawandengannilaifhitungf
tabeldantarafsignifikansisebesar : tabeldantarafsignifikansisebesar
variabelhard : variabelhard
dansoft : dansoft
skillmempunyaipengaruh : skillmempunyaipengaruh
sama

joan : joan
oki : oki
aprilyanto : aprilyanto
penelitihan : penelitihan
pelatian : pelatian
terhapa : terhapa
gondang : gondang
outbound : outbound
wego : wego
destinasisecara : destinasisecara
dipikul : pikul
perusahaanorganisasi : perusahaanorganisasi
bhakti : bhakti
bumd : bumd
ulum : ulum
abel : abel
ditoko : toko
slovinanalisis : slovinanalisis
bresponden : bresponden
berdasaekan : berdasaekan
fani : fani
rachma : rachma
noerlila : noerlila
isa : isa
anshori : anshori
fanomena : fanomena
menyayangkan : sayang
tempuh : tempuh
destination : destination
jaddih : jaddih
tersebar : sebar
menunjukkanbahwasannya : menunjukkanbahwasannya
kelamahan : kelamahan
itemitem : itemitem
holistik : holistik
penggambaran : gambar
emosi : emosi
membangkitkan : bangkit
santai : santai
waswas : waswas
tenang : tenang
keamanannya : aman
distribution : distribution
diproduksi : produksi
katun : katun
logo : logo
memperbarui : baru
assfak : assfak
assyuhada : assyuhada
tema : tema
kekinian : kini
mengang

Pandas Apply:   0%|          | 0/1019 [00:00<?, ?it/s]

0       [abstrak, teliti, metode, kuantitatif, tekan, ...
1       [aplikasi, nyata, manfaat, teknologi, informas...
2       [tuju, teliti, identifikasi, variabelvariabel,...
3       [pln, badan, usaha, milik, negara, gerak, bida...
4       [teliti, tuju, pengaruh, faktorfaktor, stres, ...
                              ...                        
1014    [umi, kholidah, pengaruh, arus, kas, operasi, ...
1015    [abstrak, ra, fajria, nurul, islami, pengaruh,...
1016    [riska, utami, ningsih, pengaruh, milik, insti...
1017    [dasar, hasil, teliti, nilai, square, besar, p...
1018    [laura, monica, analisis, pengaruh, likuiditas...
Name: Abstraksi_tokens_stemmed, Length: 1019, dtype: object


In [54]:
data_berita

,Judul,Penulis,Dospem 1,Dospem 2,Abstraksi,Abstraction,Link Download,Abstraksi_tokens,Abstraksi_tokens_fdist,Abstraksi_tokens_WSW,Abstraksi_normalized,Abstraksi_tokens_stemmed
0,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Hendri Wahyudi Prayitno,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",abstrak penelitian ini menggunakan metode kuan...,Abstract\r\nThis research use quantitative met...,https://pta.trunojoyo.ac.id/uploads/journals/0...,"[abstrak, penelitian, ini, menggunakan, metode...","{'abstrak': 1, 'penelitian': 4, 'ini': 7, 'men...","[abstrak, penelitian, metode, kuantitatif, men...","[abstrak, penelitian, metode, kuantitatif, men...","[abstrak, teliti, metode, kuantitatif, tekan, ..."
1,Pengukuran Website Quality Pada Situs Sistem A...,Muhammad Zakaria Utomo,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",aplikasi nyata pemanfaatan teknologi informasi...,Academic portal system in University of Trunoj...,https://pta.trunojoyo.ac.id/uploads/journals/1...,"[aplikasi, nyata, pemanfaatan, teknologi, info...","{'aplikasi': 1, 'nyata': 1, 'pemanfaatan': 1, ...","[aplikasi, nyata, pemanfaatan, teknologi, info...","[aplikasi, nyata, pemanfaatan, teknologi, info...","[aplikasi, nyata, manfaat, teknologi, informas..."
2,"PENGARUH PERSEPSI MANFAAT, PERSEPSI KEMUDAHAN,...",Ahmad,"Bambang Setiyo Pambudi, S.E., MM.","Fathor AS, S.E., MM.",tujuan penelitian ini adalah untuk mengidentif...,The purpose of this research was to identify v...,https://pta.trunojoyo.ac.id/uploads/journals/0...,"[tujuan, penelitian, ini, adalah, untuk, mengi...","{'tujuan': 1, 'penelitian': 4, 'ini': 3, 'adal...","[tujuan, penelitian, mengidentifikasi, variabe...","[tujuan, penelitian, mengidentifikasi, variabe...","[tuju, teliti, identifikasi, variabelvariabel,..."
3,PENGARUH PENGEMBANGAN KARIR TERHADAP SEMANGAT ...,Danial Aziz M.YD,"Dr. Hj. Iriani Ismail, Dra.,MM","Drs. Ec. Mudji Kuswinarno, M.Si",pln merupakan badan usaha milik negara yang be...,PLN is a state-owned enterprise dealy with the...,https://pta.trunojoyo.ac.id/uploads/journals/0...,"[pln, merupakan, badan, usaha, milik, negara, ...","{'pln': 5, 'merupakan': 2, 'badan': 1, 'usaha'...","[pln, badan, usaha, milik, negara, bergerak, b...","[pln, badan, usaha, milik, negara, bergerak, b...","[pln, badan, usaha, milik, negara, gerak, bida..."
4,PENGARUH FAKTOR-FAKTOR STRES KERJA TERHADAP DI...,Saiful Rahman,"Dr. Muhammad Alkirom Wildan,S.E.,M.Si","Helmi Buyung Aulia,S.ST.,S.E.,M.MT",penelitian ini bertujuan untuk mengetahui peng...,\r\n\r\nThis study aimed to determine the effe...,https://pta.trunojoyo.ac.id/uploads/journals/0...,"[penelitian, ini, bertujuan, untuk, mengetahui...","{'penelitian': 2, 'ini': 1, 'bertujuan': 1, 'u...","[penelitian, bertujuan, pengaruh, faktorfaktor...","[penelitian, bertujuan, pengaruh, faktorfaktor...","[teliti, tuju, pengaruh, faktorfaktor, stres, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1014,"Pengaruh Arus Kas Operasi, Arus Kas Investasi,...",UMI KHOLIDAH,"Echsan Gani, S.E., M.Si",NaN,umi kholidah pengaruh arus kas operasi arus ka...,"\nUmi Kholidah, The Effect of Operating Cash F...",https://pta.trunojoyo.ac.id/uploads/journals/1...,"[umi, kholidah, pengaruh, arus, kas, operasi, ...","{'umi': 1, 'kholidah': 1, 'pengaruh': 2, 'arus...","[umi, kholidah, pengaruh, arus, kas, operasi, ...","[umi, kholidah, pengaruh, arus, kas, operasi, ...","[umi, kholidah, pengaruh, arus, kas, operasi, ..."
1015,Pengaruh Self Efficacy Dan Locus Of Control Te...,R.A Fajria Nurul Islami,"Drs. Ec. Mudji Kuswinarno, M.Si",NaN,abstrak ra fajria nurul islami pengaruh self e...,"Abstract\nR.A Fajria Nurul Islami, 16021110023...",https://pta.trunojoyo.ac.id/uploads/journals/1...,"[abstrak, ra, fajria, nurul, islami, pengaruh,...","{'abstrak': 1, 'ra': 1, 'fajria': 1, 'nurul': ...","[abstrak, ra, fajria, nurul, islami, pengaruh,...","[abstrak, ra, fajria, nurul, islami, pengaruh,...","[abstrak, ra, fajria, nurul, islami, pengaruh,..."
1016,

In [55]:
data_berita['Abstraksi']

0       abstrak penelitian ini menggunakan metode kuan...
1       aplikasi nyata pemanfaatan teknologi informasi...
2       tujuan penelitian ini adalah untuk mengidentif...
3       pln merupakan badan usaha milik negara yang be...
4       penelitian ini bertujuan untuk mengetahui peng...
                              ...                        
1014    umi kholidah pengaruh arus kas operasi arus ka...
1015    abstrak ra fajria nurul islami pengaruh self e...
1016    riska utami ningsih pengaruh kepemilikan insti...
1017    berdasarkan hasil penelitian ini menunjukkan b...
1018    laura monica analisis pengaruh likuiditas kual...
Name: Abstraksi, Length: 1019, dtype: object

# K-Means Clustering

In [58]:
vectorizer = TfidfVectorizer(stop_words='english')
berita = []
for data in data_berita['Abstraksi']:
    isi = ''
    for term in data:
        isi += term + ' '
    berita.append(isi)

vectorizer.fit(data_berita)
X = vectorizer.fit_transform(data_berita)

true_k = 5
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 dospem
 penulis
 link
 judul
 download
 abstraksi_tokens_wsw
 abstraksi_tokens_stemmed
 abstraksi_tokens_fdist
 abstraksi_tokens
 abstraksi_normalized
Cluster 1:
 abstraksi_tokens
 penulis
 link
 judul
 download
 dospem
 abstraksi_tokens_wsw
 abstraksi_tokens_stemmed
 abstraksi_tokens_fdist
 abstraksi_normalized
Cluster 2:
 judul
 abstraksi_tokens_wsw
 abstraksi_tokens_stemmed
 abstraksi_tokens_fdist
 abstraksi_normalized
 abstraction
 link
 download
 penulis
 dospem
Cluster 3:
 penulis
 link
 judul
 download
 dospem
 abstraksi_tokens_wsw
 abstraksi_tokens_stemmed
 abstraksi_tokens_fdist
 abstraksi_tokens
 abstraksi_normalized
Cluster 4:
 abstraksi
 penulis
 link
 judul
 download
 dospem
 abstraksi_tokens_wsw
 abstraksi_tokens_stemmed
 abstraksi_tokens_fdist
 abstraksi_tokens
